In [ ]:
import os
import pandas as pd

# Path to the directory containing the CSV files
directory_path = '/data1-8tb-blk/sp2023stock/Insight-Wave/TopicModeling/result/integrationOutput/id/thunhoon_ids.csv'

articles_list = {}

df = pd.read_csv(directory_path)
articles_list = df['ID'].tolist()

print(len(articles_list))


In [30]:
import os
import csv
import re
import pandas as pd
import chardet
import datetime
from bs4 import BeautifulSoup


def htmlInput(file_path):
    with open(file_path, "r", encoding='utf-8') as file:
        html_content = file.read()
    soup = BeautifulSoup(html_content, "html.parser")
    return soup


#Extract title
def extract_title(soup):
    title = soup.title.string.strip() if soup.title else None
    return title

#Extract body
def extract_body(soup):
    article_divs = soup.find_all("div", class_="article-p")
    articles = []
    if article_divs:
        for article_div in article_divs:
            for p in article_div.find_all('p', {'style': 'text-align:right'}):  # if <div> contains <p style="text-align:right">
                p.decompose()  # remove <p> tag
            articles.append(article_div.get_text(strip=True))
    
        articles = [s.replace('\n', ' ') for s in articles]
        articles = [s.replace('\xa0', ' ') for s in articles]
        articles = [item for item in articles if item != '']

    
    return articles


#Extract tags
def extract_tags(soup):
    tagcloud_span = soup.find("h6", class_="newsTagColor")
    
    if tagcloud_span is not None:
        tags = tagcloud_span.find_all('a')
        tag_content = [tag.get_text(strip=True) for tag in tags]
        tag_content = [item for item in tag_content if item != '']
    else:
        tag_content = None
    
    return tag_content

def extract_pubdate(soup):
    month_list = {
        'มกราคม': '01', 
        'กุมภาพันธ์': '02', 
        'มีนาคม': '03', 
        'เมษายน': '04', 
        'พฤษภาคม': '05', 
        'มิถุนายน': '06', 
        'กรกฎาคม': '07', 
        'สิงหาคม': '08', 
        'กันยายน': '09', 
        'ตุลาคม': '10', 
        'พฤศจิกายน': '11', 
        'ธันวาคม': '12'
    }
    
    # Assuming soup is already defined with your HTML content
    date_time_div = soup.find("p", class_=re.compile("^Article__PublishTimeStamp-sc-18ssv56-0"))
    
    if date_time_div is not None:
        date_time_string = date_time_div.get_text(strip=True)
    
        # Split the string into words
        words = date_time_string.split()
    
        # Identify the components
        day = words[0]
        month = month_list.get(words[1])  # Get the numeric month
        year = words[2]
        time = words[4]
    
        # Form the date and time in the desired format
        pubdate = f"{year}-{month}-{day} {time}"
    else:
        pubdate = None
    
    return pubdate

#Extract title
def extract_intro(soup):
    intro_div = soup.find("h2", class_="entry-sub-title")
    intro = ""
    if intro_div:
        intro = intro_div.get_text(strip=True)
    else:
        intro = None

    return intro

def extract_urlPic(soup):
    # Try to find the primary image first
    primary_div = soup.find('div', class_="container")
    if primary_div:
        primary_img = primary_div.find('img')
        if primary_img:
            primary_src = primary_img.get('src')
            # Check if the src length is less than 40
            if len(primary_src) >= 40:
                return primary_src
            else:
                # If the primary src is too short, look for the alternative
                alternative_div = soup.find('div', class_="css-a1g4gz e1xtbsxb1")
                if alternative_div:
                    alternative_img = alternative_div.find('img')
                    if alternative_img:
                        alternative_src = alternative_img.get('src')
                        return alternative_src
                    else:
                        print("No alternative img tag found.")
                        return None
                else:
                    print("No alternative div with class 'css-a1g4gz e1xtbsxb1' found.")
                    return None
        else:
            print("No img tag found inside the primary div.")
            return None
    else:
        print("No primary div with class 'container' found.")
        return None

# Usage
# Assuming 'soup' has been defined and loaded with your HTML content
image_url = extract_urlPic(soup)
print(image_url)



for article_id in range(start_id, end_id+1):
    file_path = os.path.join("E:/Crimson_News/DataSet/thunhoon/article/" + str(article_id) +"/index.txt")
    
    if os.path.exists(file_path):
        try:
            soup = htmlInput(file_path)
            title = extract_title(soup)
            intro = extract_intro(soup)
            articles = extract_body(soup)
            pubdate = extract_pubdate(soup)
            tag_content = extract_tags(soup)
            url = extract_urlPic(soup)
            
            data_dict = {'Title': title, 'Intro': intro, 'Article': articles, 'DateTime': pubdate, 'Tags': tag_content, 'url_picture': url}
            
        
            with open(os.path.join("E:/Crimson_News/DataSet/thunhoon/article/" + str(article_id) +"/parsing.txt"), 'w', encoding="utf-8") as f:
                for key, value in data_dict.items():
                    if value is not None:
                        if key == 'Tags' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for tags in value:
                                f.write(f"{tags}\n")

                        elif key == 'Article' and isinstance(value, list):
                            f.write(f"[::{key}::]\n")
                            for bodys in value:
                                if bodys == 'SPONSORED':
                                    continue
                                else:
                                    f.write(f"{bodys}\n")
                                    f.write("\n")
                        else:
                            f.write(f"[::{key}::]\n{value}\n")


            with open('parsing_progress.txt', 'w') as f:
                f.write(f'{article_id}')

            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Valid'
                writer.writerow([article_id, validation])

                    
        except Exception as e:
            print(f"Error processing article {article_id}: {e}")
            with open('parsing_number.csv', 'a', newline='') as f:
                writer = csv.writer(f)
                validation = 'Error'
                writer.writerow([article_id, validation])
                continue


# Write the last processed article_id to 'csv_progress.txt' after loop ends
with open('parsing_progress.txt', 'w') as f:
    f.write(str(article_id) + '\n')

Error processing article 277352: 'charmap' codec can't decode byte 0x81 in position 548: character maps to <undefined>
Error processing article 278401: 'charmap' codec can't decode byte 0x81 in position 548: character maps to <undefined>
